In [4]:
from patra_agent.graph import run_patra_graph
from langsmith import Client
from langsmith.wrappers import wrap_openai
from langsmith.evaluation import evaluate
from langsmith import Client
from langsmith.schemas import Run, Example
from langsmith.utils import LangSmithError
from langchain_core.output_parsers import StrOutputParser
from langsmith.evaluation import evaluate, LangChainStringEvaluator
from langchain.schema.runnable import RunnablePassthrough
from patra_agent.query_responses import query_dataset

In [5]:
client = Client()
dataset_name = "patra_agent_db05"

In [6]:

dataset = client.create_dataset(dataset_name, description="patra checks")
print(f"Created new dataset: {dataset_name}")
try:
    dataset = client.read_dataset(dataset_name=dataset_name)
    print(f"Using existing dataset: {dataset_name}")
except LangSmithError:
    dataset = client.create_dataset(dataset_name, description="battle prompts")
    print(f"Created new dataset: {dataset_name}")

for qr in query_dataset:
    try:
        client.create_example(
            inputs = qr["inputs"],
            outputs= qr["outputs"],
            dataset_id=dataset.id
        )
        print(f"Added example: {qr['inputs']['question']}")
    except LangSmithError as e:
        print(f"Failed to add example: {qr['inputs']['question']}. Error: {str(e)}")
print(f"Finished adding examples to the dataset")

Created new dataset: patra_agent_db05
Using existing dataset: patra_agent_db05
Added example: How many models are there in the system?
Added example: How many edge devices are there in the system?
Added example: What is the framework was used for model with model ID 888a1c63-2d03-478d-92b9-b371c752306e-model
Added example: What is the accuracy of the model with model ID 888a1c63-2d03-478d-92b9-b371c752306e-model?
Added example: What are the 5 models most closely resembling the one with ID e835e0f3-3c13-4ab8-9b17-a8c5ea8dc788?
Added example: What is the architecture of edge device?
Added example: Does the edge device with Device ID: jetson-nano-1 support AI operations ?
Added example: What type of model is it?
Added example: What is the name of the model with model ID 888a1c63-2d03-478d-92b9-b371c752306e-model?
Added example: Who is the owner of the model with model ID 888a1c63-2d03-478d-92b9-b371c752306e-model?
Added example: Who owns most of the models?
Added example: Who is the autho

In [11]:
output_parser = StrOutputParser()

def predict(inputs:dict) ->dict:
    query = inputs["question"]
    result = run_patra_graph(query)
    return result

chain = RunnablePassthrough() | predict | output_parser


def must_mention(run: Run, example: Example) -> dict:
    prediction = run.outputs.get("output") or ""
    required = example.outputs.get("must_mention") or []
    score = all(phrase in prediction for phrase in required)
    return {"key": "must_mention", "score": score}

evaluators = [
  LangChainStringEvaluator("cot_qa"),
  LangChainStringEvaluator("labeled_criteria", config={"criteria": "conciseness"}),
  LangChainStringEvaluator("labeled_criteria", config={"criteria": "helpfulness"}),
  LangChainStringEvaluator("labeled_criteria", config={"criteria": "detail"})
]
experiment_results = evaluate(
    chain.invoke, 
    data=dataset_name, 
    evaluators=evaluators, 
    experiment_prefix="rasengan-generate01", 
    
)

View the evaluation results for experiment: 'rasengan-generate01-d64b6023' at:
https://smith.langchain.com/o/772db1be-0d45-5c99-ad5e-b5970a468c06/datasets/1dc49c6f-e953-400c-86fb-2ed55147b6b6/compare?selectedSessions=a11507b9-e1bb-412f-8883-f37d8873e637




0it [00:00, ?it/s]

================================ Human Message =================================

Name the datasheet?================================ Human Message =================================

How many deployements are made for model ID 888a1c63-2d03-478d-92b9-b371c752306e-model?

================================ Human Message =================================

Name the edge device in the system?
================================ Human Message =================================

How many bias analysis are present in UCI Adult Data Analysis?
================================ Human Message =================================

Which bias analysis has the lowest demographic parity difference?
================================ Human Message =================================

Name the model cards are present in the system?
================================ Human Message =================================

How many model cards are present in the system?
================================ Human Message ==========

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: HAS_DEPLOYMENT)} {position: line: 1, column: 77, offset: 76} for query: "MATCH (m:Model {model_id: '888a1c63-2d03-478d-92b9-b371c752306e-model'})-[r:HAS_DEPLOYMENT]-(depl:Deployment) RETURN COUNT(depl) AS NumberOfDeployments"



Invoking: `execute_cypher` with `{'query': "MATCH (m:Model {model_id: '888a1c63-2d03-478d-92b9-b371c752306e-model'})-[r:HAS_DEPLOYMENT]-(depl:Deployment) RETURN COUNT(depl) AS NumberOfDeployments"}`


The database used for data storage is the UCI Machine Learning Repository.

> Finished chain.
================================== Ai Message ==================================
Name: db_executor

The database used for data storage is the UCI Machine Learning Repository.
================================ Human Message =================================

What is the name of the model with model ID 888a1c63-2d03-478d-92b9-b371c752306e-model?


+++++++++++++++++++++++
Running Cypher Query:
[{{'NumberOfDeployments': 0}}]
+++++++++++++++++++++++
+++++++++++++++++++++++


[{{'NumberOfDeployments': 0}}]
Invoking: `execute_cypher` with `{'query': "MATCH (m:Model {model_id: '3075715e-bac0-4783-bff9-4d8ea6a5f638'})-[r:USED]-(mc:ModelCard)-[r2:XAI_ANALYSIS]-(ea:ExplainabilityAnalysis) RETURN ea;"}`






...

Error running target function: Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input '': expected an identifier (line 1, column 268 (offset: 267))
"MATCH (m:Model)-[:HAS_DEPLOYMENT]-(depl:Deployment)-[:DEPLOYED_IN]-(device:EdgeDevice {location: 'SF-zone-02'}) MATCH (mc:ModelCard)-[:USED]-(m) MATCH (mc)-[:BIAS_ANALYSIS]-(ba:BiasAnalysis) RETURN m, mc.external_id, ba ORDER BY ba.equal_odds_difference DESC LIMIT 1."
                                                                                                                                                                                                                                                                            ^}



Invoking: `execute_cypher` with `{'query': "MATCH (m:Model)-[:HAS_DEPLOYMENT]-(depl:Deployment)-[:DEPLOYED_IN]-(device:EdgeDevice {location: 'SF-zone-02'}) MATCH (mc:ModelCard)-[:USED]-(m) MATCH (mc)-[:BIAS_ANALYSIS]-(ba:BiasAnalysis) RETURN m, mc.external_id, ba ORDER BY ba.equal_odds_difference DESC LIMIT 1."}`


================================== Ai Message ==================================
Name: query_agent

MATCH (m:Model) RETURN m.model_type


> Entering new AgentExecutor chain...
The bias analysis with the lowest demographic parity difference is as follows:

- Name: 7e110269-8e1d-4a2c-b4db-c713b292d511bias_analysis
- External ID: 7e110269-8e1d-4a2c-b4db-c713b292d511-bias
- Demographic Parity Difference: -0.9818984619471309
- Equal Odds Difference: -0.3787884779765942

> Finished chain.
================================== Ai Message ==================================
Name: db_executor

The bias analysis with the lowest demographic parity difference is as follows:

- Name: 7e1102

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: VERSION_OF)} {position: line: 1, column: 71, offset: 70} for query: "MATCH (m:Model {model_id: 'e835e0f3-3c13-4ab8-9b17-a8c5ea8dc788'})-[r:VERSION_OF]-(similar:Model) RETURN similar ORDER BY similar.model_id LIMIT 5;"



Invoking: `execute_cypher` with `{'query': "MATCH (m:Model {model_id: 'e835e0f3-3c13-4ab8-9b17-a8c5ea8dc788'})-[r:VERSION_OF]-(similar:Model) RETURN similar ORDER BY similar.model_id LIMIT 5;"}`




+++++++++++++++++++++++
Running Cypher Query:
[]
+++++++++++++++++++++++
+++++++++++++++++++++++


[]There are 100 models in the system.

> Finished chain.
================================== Ai Message ==================================
Name: db_executor

There are 100 models in the system.
The number of edge devices in the system is 10.

> Finished chain.
================================== Ai Message ==================================
Name: db_executor

The number of edge devices in the system is 10.
The accuracy of the model with model ID 888a1c63-2d03-478d-92b9-b371c752306e-model is approximately 56.86%.

> Finished chain.
================================== Ai Message ==================================
Name: db_executor

The accuracy of the model with model ID 888a1c63-2d03-478d-92b9-b3

25it [00:41,  1.65s/it]
